# Leren: Programming assignment 4

**Student 1:**  <span style="color:red">Wim Berkelmans</span> (<span style="color:red">10793674</span>)<br>
**Student 2:** <span style="color:red">Philip Bouman</span> (<span style="color:red">10668667</span>)<br>

-----------------------------------

In [11]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as linalg
import scipy.ndimage
import operator
import math

# Load data from .csv 
def readData():
    training_set = np.loadtxt('digits123-1.csv', skiprows = 1, delimiter=';');
    test_set = np.loadtxt('digits123-2.csv', skiprows = 1, delimiter=';');
    return training_set, test_set

#### 1 a) K-nearest neighbour

In [30]:
# Calculate Euclidian distance between two values
def euclideanDistance(data1, data2):
    dist = 0
    for i in range(len(data1)-1):
        dist += pow((data1[i] - data2[i]), 2)
        
    return math.sqrt(dist)
    
# Get most similar neighbors
def getNeighbors(training_set, test_set, k):
    distances = []
    for i in range(len(training_set)):
        dist = euclideanDistance(training_set[i], test_set)
        distances.append((training_set[i], dist))
        
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][0])
        
    return neighbors

# Get votes for response
def getResponse(neighbors):
    classVotes = {}
    for i in range(len(neighbors)):
        response = neighbors[i][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
        sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)

    return sortedVotes[0][0]
            
# Count number of correct predictions
def getAccuracy(test_set, target):
    correct = 0
    for i in range(len(test_set)):
        if test_set[i][-1] == target[i]:
            correct += 1
            
    return (correct/float(len(test_set))) * 100.0
    
# Main method
def k_nearest_neighbor(k):
    training_set, test_set = readData()
    predictions = []
    
    for i in range(len(test_set)):
        neighbors = getNeighbors(training_set, test_set[i], k)
        result = getResponse(neighbors)
        predictions.append(result)
        
    accuracy = getAccuracy(test_set, predictions)
    print "Accuracy: ", accuracy 
    
# run with differnt k's
k_nearest_neighbor(1)
k_nearest_neighbor(2)
k_nearest_neighbor(3)

Accuracy:  98.7447698745
Accuracy:  97.9079497908
Accuracy:  97.9079497908


#### b) Weighted nearest neighbour

#### c) Predictive value